In [43]:
import os
import pandas as pd
import numpy as np
import re
import string

In [44]:
data_path = os.listdir('Datasets/Tweet Data')
data_path.sort()
data_path = ['Datasets/Tweet Data/{}'.format(file) for file in data_path]
data_path = pd.DataFrame(data_path,columns=['File Path'])

try:
    data_path = pd.concat([data_path,pd.DataFrame(data_path['File Path'].str.split(r'_',expand=True)[2])],axis=1)
    data_path = pd.concat([data_path,data_path[2].str.split(r'\.',expand=True)],axis=1,)
    data_path = data_path[['File Path',0]]
    data_path.columns = ['File Path', 'Counter']
    data_path['Counter'] = data_path['Counter'].apply(lambda x: int(x))
    
except:
    print('error')

In [45]:
from distutils.log import error


for i,file in enumerate(data_path.values):
    if np.mod(file[1],1440)==0:
        try:
            print(i,file[1],pd.read_csv(file[0],engine='python',index_col=0).shape)
        except error as e:
            print(e)

22 1440 (19357, 11)
47 2880 (20904, 11)
71 4320 (20553, 11)
96 5760 (20568, 11)
121 7200 (20319, 11)
145 8640 (20308, 11)
169 10080 (20598, 11)
193 11520 (20454, 11)
217 12960 (20477, 11)
240 14400 (20265, 11)
264 15840 (20371, 11)
289 17280 (20204, 11)
313 18720 (20368, 11)
337 20160 (20563, 11)
361 21600 (20744, 11)
384 23040 (20156, 11)
408 24480 (20535, 11)
432 25920 (20602, 11)
456 27360 (20306, 11)
481 28800 (20125, 11)
505 30240 (20490, 11)
529 31680 (20497, 11)
552 33120 (20452, 11)
577 34560 (20588, 11)
601 36000 (20356, 11)
625 37440 (20595, 11)
649 38880 (20556, 11)
673 40320 (20648, 11)
697 41760 (20663, 11)
721 43200 (20452, 11)
743 44640 (20173, 11)
744 44640 (20187, 11)
766 46080 (20411, 11)
790 47520 (20572, 11)
813 48960 (20270, 11)
837 50400 (20156, 11)
861 51840 (20398, 11)
885 53280 (20335, 11)
908 54720 (19922, 11)
931 56160 (19966, 11)


- Remove @[someone]
- Remove string 'RT'
- Remove punctuation (Remove # only; keep the string)
- Remove New Lines
- Remove links to http

In [46]:
def pre_process_tweet (text):
    #Remove all links starting with http...
    text = re.sub('https?:\/\/.*[\r\n]*','',text)
    #Remove RT
    text = re.sub('^RT[\s]+','',text)
    #Remove @[User]
    text = re.sub('@[^ ]+', '', text)
    #Remove Punctuation first
    text = text.translate(str.maketrans('','', string.punctuation))
    #Convert text to lower case
    text = text.lower()
    return text

In [47]:
unproc_data = pd.read_csv(data_path['File Path'][0],index_col=0)

In [48]:
print(unproc_data['text'][0])

📈Analysis June 01th📈

The current price of $BTC is 31817.14$ (⬆0.35%), and its dominance is 41.73%. 

The market sentiment is at 17 points [💀Extreme Fear]. 

Total #crypto market cap is 💰1.45 trillion $💰 approx.

#bitcoin https://t.co/EihSOAU9jp


In [51]:
pre_process_tweet(unproc_data['text'][0])

'📈analysis june 01th📈\n\nthe current price of btc is 3181714 ⬆035 and its dominance is 4173 \n\nthe market sentiment is at 17 points 💀extreme fear \n\ntotal crypto market cap is 💰145 trillion 💰 approx\n\nbitcoin '